# weatheri 2022년 일자별 데이터 크롤링

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import asyncio
import aiohttp

In [4]:
url = r'https://www.weatheri.co.kr/bygone/bygone01.php'
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
headers = { 'User-Agent' : user_agent }

In [5]:
def YYYYMMDD_to_time(date_str):
    year = int(date_str[:4])
    month = int(date_str[4:6])
    day = int(date_str[6:])
    return pd.Timestamp(year, month, day)